In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('insurance_data-2.csv')
df.head()

,age,affordability,insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df[['age','affordability']],df.insurance,test_size=0.2,random_state=25)

In [5]:
X_train

,age,affordability
96,48,0
366,28,0
491,46,0
315,40,1
488,54,1
...,...,...
317,56,1
143,58,0
474,36,0
318,23,1


In [6]:
len(X_train)

400

In [7]:
df.shape

(500, 3)

In [8]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100
X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100

In [9]:
X_train_scaled#to_bring_age_affordability_on_the_same_scale

,age,affordability
96,0.48,0
366,0.28,0
491,0.46,0
315,0.40,1
488,0.54,1
...,...,...
317,0.56,1
143,0.58,0
474,0.36,0
318,0.23,1


In [13]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(
optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train_scaled,y_train,epochs=5000)

Epoch 1/5000
13/13 [==============================] - 0s 510us/step - loss: 0.7956 - accuracy: 0.5175
Epoch 2/5000
13/13 [==============================] - 0s 269us/step - loss: 0.7910 - accuracy: 0.5175
Epoch 3/5000
13/13 [==============================] - 0s 264us/step - loss: 0.7866 - accuracy: 0.5175
Epoch 4/5000
13/13 [==============================] - 0s 257us/step - loss: 0.7820 - accuracy: 0.5175
Epoch 5/5000
13/13 [==============================] - 0s 265us/step - loss: 0.7780 - accuracy: 0.5175
Epoch 6/5000
13/13 [==============================] - 0s 244us/step - loss: 0.7737 - accuracy: 0.5175
Epoch 7/5000
13/13 [==============================] - 0s 325us/step - loss: 0.7700 - accuracy: 0.5175
Epoch 8/5000
13/13 [==============================] - 0s 320us/step - loss: 0.7662 - accuracy: 0.5175
Epoch 9/5000
13/13 [==============================] - 0s 264us/step - loss: 0.7629 - accuracy: 0.5175
Epoch 10/5000
13/13 [==============================] - 0s 309us/step - loss: 0.759

In [14]:
model.evaluate(X_test_scaled,y_test)

4/4 [==============================] - 0s 662us/step - loss: 0.6954 - accuracy: 0.4900


[0.6954419016838074, 0.49000000953674316]

In [15]:
X_test_scaled

,age,affordability
411,0.62,1
380,0.40,0
38,0.38,0
254,0.59,1
234,0.53,0
...,...,...
135,0.27,1
282,0.54,1
48,0.50,1
176,0.46,0


In [26]:
y_predcited=model.predict(X_test_scaled)

4/4 [==============================] - 0s 678us/step


In [17]:
y_test

411    1
380    0
38     0
254    0
234    0
      ..
135    1
282    1
48     0
176    0
133    1
Name: insurance, Length: 100, dtype: int64

In [19]:
#for_knowing_individual_weights_and_bias
coef, intercept= model.get_weights()
coef, intercept

(array([[1.3474427 ],
        [0.01829723]], dtype=float32),
 array([-0.48462254], dtype=float32))

In [20]:
#Implement_all_this_from_scratch
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [21]:
def prediction_function(age,affordability):
    weighted_sum=coef[0]*age+ coef[1]*affordability+ intercept
    return sigmoid(weighted_sum)
    

In [22]:
prediction_function(0.62,1)

0.5912388753858494

In [25]:
prediction_function(0.40,0)

0.5135852925600509

In [27]:
#implement_gradient_descent_function_from_scratch
def log_loss(y_true,y_predicted):
    epsilon=1e-15
    y_predicted_new=[max(i,epsilon) for i in y_predicted]
    y_predicted_new=[min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new=np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))
    

In [29]:
#New_sigmoid_function,_so_that_instead_of_taking_one_value_it_is_taking_array_of_values
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [33]:
def gradient_descent(age, affordability, y_true, epochs,loss_thresold):
    #w1,w2,bias
    w1=w2=1
    bias=0
    rate=0.5
    n=len(age)
    for i in range(epochs):
        weighted_sum=w1*age + w2*affordability + bias
        y_predicted=sigmoid_numpy(weighted_sum)
        loss=log_loss(y_true, y_predicted)
        w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d=(1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
        bias_d=np.mean(y_predicted-y_true)
        w1=w1-rate*w1d
        w2=w2-rate*w2d
        bias=bias-rate*bias_d
        print(f'Epoch{i},w1:{w1},w2:{w2},bias:{bias},loss={loss}')
        if loss<=loss_thresold:
            break
    return w1,w2,bias
    

In [36]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordability'],y_train,1000,0.6881)

Epoch0,w1:0.96404256522264,w2:0.928102135511628,bias:-0.09256779203486516,loss=0.7977143087827275
Epoch1,w1:0.9336783245951066,w2:0.8638106865239293,bias:-0.17120046082101087,loss=0.7696647371680925
Epoch2,w1:0.9084128486466874,w2:0.8067409811878713,bias:-0.23715650938211894,loss=0.7487877688169594
Epoch3,w1:0.8876861369429523,w2:0.7563230255169757,bias:-0.29184776783377653,loss=0.7335245567389435
Epoch4,w1:0.8709207647017486,w2:0.7118841329480128,bias:-0.3367208020970086,loss=0.722514248044111
Epoch5,w1:0.8575581062673359,w2:0.67271886686048,bias:-0.37316869978979245,loss=0.7146417177433135
Epoch6,w1:0.8470815395234046,w2:0.6381395006816456,bias:-0.402475361972943,loss=0.7090366601685997
Epoch7,w1:0.8390285218296613,w2:0.6075073448858407,bias:-0.4257872283476168,loss=0.7050445875887994
Epoch8,w1:0.8329945360827543,w2:0.5802487786617468,bias:-0.4441048278898485,loss=0.7021868797151976
Epoch9,w1:0.8286318026745338,w2:0.5558605971145563,bias:-0.45828694738629056,loss=0.7001207486552257
E

(1.2876930470107628, 0.021743225384722994, -0.46026915099350024)

In [37]:
coef,intercept

(array([[1.3474427 ],
        [0.01829723]], dtype=float32),
 array([-0.48462254], dtype=float32))